# Evaluation

In [ ]:
from mqt.problemsolver.partialcompiler.qaoa import QAOA
from mqt.problemsolver.partialcompiler.evaluator import evaluate_QAOA
import numpy as np
from time import time
from mqt.qcec import verify

In [ ]:
q = QAOA(num_qubits=3, repetitions=1, remove_probability=0.7, considered_following_qubits=100)
qc_compiled_with_all_gates = q.qc_compiled.copy()

print("Start New")
start = time()
compiled_qc = q.check_gates(
    qc=qc_compiled_with_all_gates,
    optimize_swaps=True
)
time_new_scheme = time() - start

print("Start Baseline")
start = time()
qc_baseline_compiled = q.compile_qc(baseline=True, opt_level=2)
time_baseline = time() - start

time_ratio = time_new_scheme / time_baseline
if qc_baseline_compiled.count_ops().get("cx"):
    cx_count_ratio = compiled_qc.count_ops()["cx"] / qc_baseline_compiled.count_ops()["cx"]
else:
    cx_count_ratio = 0
# print("QCEC:", verify(q.qc_baseline, qc_baseline_compiled))
print (np.round(time_ratio,5), np.round(cx_count_ratio,3))

In [ ]:
print(verify(compiled_qc, qc_baseline_compiled))

# Paper Example

In [ ]:
from qiskit import transpile, QuantumCircuit
from qiskit.providers.fake_provider import FakeQuito, FakeManila
from qiskit.circuit import Parameter
cmap = FakeQuito().configuration().coupling_map
cmap
p = Parameter("p")

In [ ]:
qc = QuantumCircuit(4)
qc.rzz(p, 0,1)
qc.rzz(p, 0,2)
qc.rzz(p, 0,3)
qc.rzz(p, 1,2)
qc.rzz(p, 1,3)
qc.rzz(p, 2,3)
qc.draw()

In [ ]:
qc = transpile(qc, coupling_map=cmap, optimization_level=3)
print(qc.draw())
print(qc.count_ops()["swap"])

In [ ]:
qc = QuantumCircuit(4)
qc.rzz(p, 0,1)
qc.rzz(p, 0,2)
#qc.rzz(p, 0,3)
qc.rzz(p, 1,2)
#qc.rzz(p, 1,3)
qc.rzz(p, 2,3)
qc.draw()
qc = transpile(qc, coupling_map=cmap, optimization_level=3)
print(qc.draw())
print(qc.count_ops()["swap"])

# Proof of Concept

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import CXGate
from qiskit.circuit import Parameter
from qiskit.providers.fake_provider import FakeManila, FakeMontreal, FakeWashington
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import CXCancellation, CommutativeCancellation, CommutativeInverseCancellation, ConsolidateBlocks, Collect2qBlocks
from numpy.random import random
from time import time
from qiskit_trebugger import Debugger

In [ ]:
p = Parameter("alpha")
num_qubits = 120
backend = FakeWashington()
qc = QuantumCircuit(num_qubits)
qc_baseline = QuantumCircuit(num_qubits)
remove_gates = []
for i in range(num_qubits):
    for j in range(i+1, min(num_qubits, i+3)):
        qc.rzz(p,i,j)
        if random()<0.5:
            remove_gates.append(True)
        else:
            remove_gates.append(False)
            qc_baseline.rzz(p,i,j)
qc_compiled_origin = transpile(qc, backend=backend, optimization_level=2)
rz_count_before = qc_compiled_origin.count_ops()["rz"]

In [ ]:
def set_to_be_checked_gates(qc_compiled, to_be_checked_gates, remove_gates):
    for i in range(len(remove_gates)):
        if remove_gates[i]:
            qc_compiled._data.remove(to_be_checked_gates[i])

def set_to_be_checked_indices(qc_compiled, to_be_checked_gates, remove_gates):
    offset = 0
    for i in range(len(remove_gates)):
        if remove_gates[i]:
            del qc_compiled._data[to_be_checked_gates[i-offset]]
            offset+=1

In [ ]:
def get_indices_of_relevant_gates(qc:QuantumCircuit):
    indices = []
    for i, gate in enumerate(qc._data):
        if gate.operation.name == "rz" and isinstance(gate.operation.params[0], Parameter):
            indices.append(i)
    return indices

In [ ]:
qc_compiled = qc_compiled_origin.copy()
to_be_checked_indices = get_indices_of_relevant_gates(qc_compiled)
to_be_checked_gates = [qc_compiled._data[i] for i in to_be_checked_indices]
assert len(to_be_checked_gates) == len(remove_gates)

start = time()
set_to_be_checked_gates(qc_compiled, to_be_checked_gates, remove_gates=remove_gates)
#set_to_be_checked_indices(qc_compiled, to_be_checked_indices, remove_gates=remove_gates)
print("Before Opt. Run:", qc_compiled.count_ops())
transpile_passes = [    
    CommutativeCancellation(),
    CommutativeInverseCancellation(),
    CXCancellation(),
    Collect2qBlocks(),
    ConsolidateBlocks(),
]
pm = PassManager(transpile_passes)
#qc_compiled = pm.run(qc_compiled).decompose()
#qc_compiled = transpile(qc_compiled,optimization_level=3, basis_gates=backend.configuration().basis_gates) #reduced swaps
#Debugger.debug(qc_compiled,optimization_level=3, basis_gates=backend.configuration().basis_gates)
print("After Opt. Run:", qc_compiled.count_ops())
new_comp_time = time()-start
print("new_comp_time:", new_comp_time)

cx_count_new_scheme = qc_compiled.count_ops()["cx"]
#rz_count_new_scheme = qc_compiled.count_ops()["rz"]
#assert rz_count_before-rz_count_new_scheme== sum(remove_gates)

# Comparison to Baseline

In [ ]:
start = time()
baseline_compiled = transpile(qc_baseline, backend=backend, optimization_level=2)
regular_comp_time = time()-start
cx_count_all_offline = baseline_compiled.count_ops()['cx']

In [ ]:
time_relation = new_comp_time/regular_comp_time
cx_relation = cx_count_new_scheme/cx_count_all_offline

In [ ]:
print(new_comp_time, regular_comp_time, time_relation)
print(cx_count_new_scheme, cx_count_all_offline, cx_relation)

In [ ]:
from time import time

In [ ]:
x = list(range(10000))
start = time()
for i in range (2000):
    del x[i+1000]
print(time()-start)

In [ ]:
x = list(range(10000))
start = time()
del x[1000:3000]
print(time()-start)